In [14]:
import os, pandas as pd, numpy as np, random,gc
import copy, cv2
pd.options.mode.chained_assignment = None
import torch, torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [15]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [16]:
# df_test_01 = pd.read_csv('./csv_pickle/df_224_sz384_0.csv')
# df_test_02 = pd.read_csv('./csv_pickle/df_224_sz384_1.csv')
# df_test_03 = pd.read_csv('./csv_pickle/df_224_sz384_2.csv')
# df_test_04 = pd.read_csv('./csv_pickle/df_224_sz384_3.csv')
# df = pd.concat([df_test_01,df_test_02,df_test_03,df_test_04])
# df.to_csv('./csv_pickle/df_224_embed_sz_384_146server.csv', encoding='utf-8-sig', index=False)

In [17]:
# # fix path
# df_224_3d = pd.read_csv("./csv_pickle/df_224_embed.csv")
# ct_path_stack = []
# for old_path in df_224_3d['ct_path'].tolist():
#     old_ = '/ssd2/COVID2023_data_embed_npy/'
#     new_ = '/ssd8/2023COVID19/2023_covid/COVID2023_data_embed_npy/'
#     new_path = old_path.replace(old_, new_)
#     ct_path_stack.append(new_path)
# df_224_3d['ct_path'] = ct_path_stack
# df_224_3d.to_csv("./csv_pickle/df_224_embed_146server.csv", index=False, encoding='utf-8-sig')

In [18]:
# 修改init讀取全部embed的方法（記憶體無法負擔所有npy資料），改以呼叫index階段時再進行讀取
class COVID_Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, data_split, ct_len_s=50 , transform = None):
        if data_split == 'test':
            dataset = pd.read_pickle(csv_path)
            self.embed_info = dataset
            classes = [999]
            classes = sorted(list(classes))
            class_to_idx = {classes[i]: i for i in range(len(classes))}
            ct_path = np.unique(dataset.iloc[:, 2])
            imgs = []
            for i_scan_dir in tqdm(ct_path):
                temp_df = dataset[dataset['ct_path'] == i_scan_dir]
                imgs.append((i_scan_dir, 999))
            
        elif data_split == 'train' or 'valid': 
            df = pd.read_csv(csv_path)
            # df['embed'] = self.load_npy(df.ct_path.values.tolist(), df.embed.values.tolist())
            dataset = df[df['split'] == data_split]
            self.embed_info = dataset
            
            classes = set(dataset['label'])
            classes = sorted(list(classes))
            class_to_idx = {classes[i]: i for i in range(len(classes))}
            
            ct_path = np.unique(dataset.iloc[:, 4])
            imgs = []
            for i_scan_dir in tqdm(ct_path):
                temp_df = dataset[dataset['ct_path'] == i_scan_dir]
                imgs.append((i_scan_dir, temp_df.iloc[0, 3]))
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.ct_len_s = ct_len_s
        self.imgs = imgs
        self.transform = transform
    def load_npy(self, npy_path, npy_file):
        new_npy_embed = []
        for path_, file_ in zip(npy_path, npy_file):
            new_npy_embed.append(np.load(os.path.join(path_, file_)))
        return new_npy_embed
    def __getitem__(self, index):
        img_scan_dir, label = self.imgs[index]
        
        label = self.class_to_idx[label]
        temp_df = self.embed_info[self.embed_info['ct_path'] == img_scan_dir]
        temp_df['embed'] = self.load_npy(temp_df['ct_path'].values.tolist(), temp_df['embed'].values.tolist())
        random.seed(4019)
        if len(temp_df) >= self.ct_len_s:
            temp_index = [x for x in range(len(temp_df))]
            target_index = random.sample(temp_index, k = self.ct_len_s)
            
        elif len(temp_df) < self.ct_len_s:
            target_index = [x for x in range(len(temp_df))]
            temp = random.choices(target_index, k = self.ct_len_s - len(target_index))
            target_index += temp
        
        target_index.sort()
        embed = temp_df.iloc[target_index, 1]
        img = []
        for i_embed in embed:
            img.append(i_embed)
        # img = np.expand_dims(np.array(img).reshape((1536, 8*8*self.ct_len_s)), axis=0)
        img = np.array(img)
        if len(img.shape)==4:
            img = np.array(img).reshape((1536, 8*8*self.ct_len_s))
        # else:
        #     img = img.reshape((img.shape[1],img.shape[0]))
        # img = np.concatenate([img,img,img], axis=0)
        # print(img.shape)
        return img, label

    def __len__(self):
        return len(self.imgs)

In [6]:
import torch,os
import torch.nn as nn
import timm
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self, ct_len=224, kernal_size = 3, pre_train=True):
        super(MyModel, self).__init__()
        # self.conv1d = nn.Conv1d(in_channels=224, out_channels=CONFIG.ct_len_get, kernel_size=kernal_size)
        self.conv1d = nn.Conv1d(in_channels=100, out_channels=CONFIG.ct_len_get, kernel_size=kernal_size)
        self.backbone = timm.create_model('resnet18', pretrained=pre_train, num_classes=1)
    def forward(self, x):
        
        x = self.conv1d(x)
        
        x = torch.cat((x.unsqueeze(1), x.unsqueeze(1), x.unsqueeze(1)), dim=1)
        
        x = self.backbone(x)
        return x

In [7]:
class CONFIG:
    # model_path1 = "/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/output/f1_best_model_k1_convembed_check_from144weight[384].bin"
    model_path1 = "/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/output/f1_best_model_k1_convslice_check_from144weight[384].bin"
    pre_train = False
    N_EPOCHS = 100
    train_batch_size = 32
    valid_batch_size = 32
    ct_len_get = 100 #100
    kernal_size = 1 #1
    SEDD =42
    LR = 3e-5 #3e-5
    WEIGHT_DECAY = 1e-3 #1e-3

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [9]:
# mean 10
def valid_one(model, loader):
    losses, predicts = [], []
    true_y=[]
    pred_y=[]
    model.eval()
    for images, label in loader:
        with torch.no_grad():
            images = images.cuda().float()
            labels = label.cuda().float()
            out = model(images)
        predicts.append(out.cpu())
        true_y.append(labels.cpu().numpy())
        pred_y.append(torch.sigmoid(out).cpu().numpy())
        
    true_y=np.concatenate(true_y)
    pred_y=np.concatenate(pred_y)
    
    gc.collect()
    
    true_y=np.array(true_y).reshape(-1,1)
    pred_y=np.array(pred_y).reshape(-1,1)

    return true_y,pred_y

In [12]:
pred_path = CONFIG.model_path1
model = MyModel(ct_len = CONFIG.ct_len_get, kernal_size=CONFIG.kernal_size, pre_train=CONFIG.pre_train).cuda()
model.load_state_dict(torch.load(pred_path))
model.cuda()
# df_path = './csv_pickle/df_224_embed_146server.csv' #144 server embedding result path
df_path = './csv_pickle/df_224_embed_sz_384_146server.csv'
print("==========data loader==========")
valid_ds = COVID_Dataset(csv_path = df_path,data_split = 'valid', ct_len_s = CONFIG.ct_len_get, transform = None)
valid_loader = DataLoader(valid_ds, batch_size=CONFIG.valid_batch_size, num_workers=15, shuffle=False, pin_memory=True)


==========data loader==========


100%|██████████| 704/704 [00:06<00:00, 105.00it/s]


In [13]:
total_pred=[]
for i in range(10):
    true_y,pred_y=valid_one(model, valid_loader)
    total_pred.append(pred_y)
# for i in range(len(total_pred)):
#     print(f1_score(np.array(true_y),np.round(total_pred[i]),average='macro'))  
tn, fp, fn, tp = confusion_matrix(np.array(true_y), np.round(np.mean(total_pred,axis=0))).ravel()
print("Mean F1-Score: {}".format(f1_score(np.array(true_y),np.round(np.mean(total_pred,axis=0)),average='macro')))
print("Negative Accuracy: {}".format(tn/(tn+fp)))
print("Positive Accuracy: {}".format(tp/(tp+fn)))

Mean F1-Score: 0.8884825444051875
Negative Accuracy: 0.9381663113006397
Positive Accuracy: 0.8297872340425532


In [ ]:
# ======================================
# model: 2dcnn(efficientnet_b3a) + 1dcnn + resnet18 (from 144 server)
# LR: (0.0001, decay:0.0005)
# batch size: 8
# image size: [384x384]
# datatype: challenge data
# (weight =  best f1 socre checkpoint)
# ------------------------------------
# 以embedding dim做1d conv
# Mean F1-Score: 0.8830530189078547
# Negative Accuracy: 0.9253731343283582
# Positive Accuracy: 0.8382978723404255
# ------------------------------------
# 以slice dim做1d conv
# Mean F1-Score: 0.8884825444051875
# Negative Accuracy: 0.9381663113006397
# Positive Accuracy: 0.8297872340425532

In [ ]:
# failed result*
# ======================================
# model: 2dcnn(efficientnet_b3a) + 1dcnn + resnet18
# LR: (0.0001, decay:0.0005)
# batch size: 8
# image size: [256x256]
# datatype: challenge data
# (weight =  best f1 socre checkpoint)
# ------------------------------------
# 以embedding dim做1d conv
# Mean F1-Score: 0.9221583370190181
# Specificity: 0.9637526652452025
# Sensitivity: 0.8680851063829788
# ------------------------------------
# 以slice dim做1d conv
# Mean F1-Score: 0.913347004465955
# Specificity: 0.9658848614072495
# Sensitivity: 0.8425531914893617

# 路徑：/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix

In [ ]:
# ======================================
# model: 2dcnn(efficientnet_b3a) + 1dcnn + resnet18
# LR: (0.0001, decay:0.0005)
# batch size: 8
# image size: [384x384]
# datatype: challenge data
# (weight =  best f1 socre checkpoint)
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=100[pretrain=True]
# Mean F1-Score: 0.8964
# Negative Accuracy: 0.9744
# Positive Accuracy: 0.7872
# ------------------------------------
# [conv axis=embedding]
# kernel=3; ct_len=100[pretrain=True]
# Mean F1-Score: 0.8964
# Negative Accuracy: 0.9744
# Positive Accuracy: 0.7872
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9221
# Negative Accuracy: 0.9637
# Positive Accuracy: 0.8680
# ------------------------------------
# [conv axis=slice]
# kernel=1; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9133
# Negative Accuracy: 0.9658
# Positive Accuracy: 0.0.8425
# ------------------------------------
# [conv axis=embedding]
# kernel=3; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9160
# Negative Accuracy: 0.9744
# Positive Accuracy: 0.8340
# ------------------------------------
# [conv axis=embedding]
# kernel=7; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9150
# Negative Accuracy: 0.9658
# Positive Accuracy: 0.8468
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=200[pretrain=False]
# Mean F1-Score: 0.9076
# Negative Accuracy: 0.9701
# Positive Accuracy: 0.8212
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=224[pretrain=False]
# Mean F1-Score: 0.9015
# Negative Accuracy: 0.9616
# Positive Accuracy: 0.8212